In [1]:
import pandas as pd
import numpy as np
import os
import argparse
from tensorflow import keras
from tensorflow.keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from utils import DATA_DIR, INPUT_SHAPE, batch_generator
import matplotlib.pyplot as plt

In [2]:
np.random.seed(0)

In [9]:
# loading data and splitting it into training testing
def load_data():
    
    df = pd.read_csv(DATA_DIR, names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])
    
    image_path = []
    steering = []
    for i in range(len(df)):
        indexed_data = df.iloc[i]
        center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
        for j in range(3):
            image_path.append(indexed_data[j])
            steering.append(float(indexed_data[3]))
            
    # setting X and y as numpy arrays, also fixing their shapes
    X = np.asarray(image_path)
    X = X.reshape(-1, 1)
    y = np.asarray(steering)
    y = y.reshape(-1, 1)   
    
    # dividing our data to 80% training and 20% validation
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)
    
    return X_train, X_valid, y_train, y_valid

In [10]:
def build_model():
    """
    NVIDIA suggested model with some modifications from naokishibuya
    """
    
    model = keras.Sequential([
        
        # normalizing the data
        Lambda(lambda x: x / 127.5 - 1.0, input_shape=INPUT_SHAPE),
        
        # 4 convolution layers, each layer is subsampled by 2x2
        Conv2D(24, (5, 5), activation='elu', strides=(2, 2)),
        Conv2D(36, (5, 5), activation='elu', strides=(2, 2)),
        Conv2D(48, (5, 5), activation='elu', strides=(2, 2)),
        Conv2D(64, (3, 3), activation='elu'),
        Conv2D(64, (3, 3), activation='elu'),
        
        #Dropout(rate=0.4),  # Nvidia suggests a dropout layer to avoid overfitting but apparently I didn't need it
        
        Flatten(),
        Dense(units=1164, activation='elu'),
        Dense(units=100, activation='elu'),
        Dense(units=50, activation='elu'),
        Dense(units=10, activation='elu'),
        
        # output layer, outputing the predicted steering angle
        Dense(units=1, activation='elu')
    ])
    
    model.compile(loss='mse',  # mean-squared-error
                  optimizer='adam')
    
    # printing the model summary to check each layer output dimentions
    model.summary()
    
    return model
    

In [11]:
def train_model(model, X_train, X_valid, y_train, y_valid):
    
    batch_size = 32
    steps_per_epoch_train = np.ceil((len(X_train)) / batch_size)
    steps_per_epoch_valid = np.ceil((len(X_valid)) / batch_size)
    number_of_epochs = 10
    
    print('X train: {}'.format(X_train.shape))
    print('X valid: {}'.format(X_valid.shape))
    print('y train: {}'.format(y_train.shape))
    print('y valid: {}'.format(y_valid.shape))
    print('steps per epoch (train): {}'.format(steps_per_epoch_train))
    print('steps per epoch (test): {}'.format(steps_per_epoch_valid))

    model.fit_generator(batch_generator(X_train, y_train, batch_size=batch_size, is_training=1),
                        steps_per_epoch=steps_per_epoch_train,
                        epochs=number_of_epochs,
                        validation_data=batch_generator(X_valid, y_valid, batch_size=batch_size, is_training=0),
                        validation_steps=steps_per_epoch_valid,
                        verbose=1)
    
    model.save('model.h5')

In [12]:
if __name__ == '__main__':
    data = load_data()
    model = build_model()
    train_model(model, *data)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1152)             